In [ ]:
import numpy as np
from PIL import Image
import heapq
from collections import Counter
from skimage.metrics import structural_similarity as ssim

print("Huffman Compression Program")
print("=================================================================")

h = int(input("Enter 1 if you want to input color image files, 2 for default grayscale case:"))

def huffman_encoding(data):
    frequency = Counter(data)
    heap = []
    for value, freq in frequency.items():
        heapq.heappush(heap, (freq, len(heap), value))

    count = len(heap)
    while len(heap) > 1:
        freq1, _count1, left = heapq.heappop(heap)
        freq2, _count2, right = heapq.heappop(heap)
        heapq.heappush(heap, (freq1 + freq2, count, (left, right)))
        count += 1

    huffman_tree = {}
    stack = [(heap[0][-1], '')] if heap else []
    while stack:
        node, code = stack.pop()
        if isinstance(node, tuple):
            stack.append((node[0], code + '0'))
            stack.append((node[1], code + '1'))
        else:
            huffman_tree[node] = code

    encoded_data = ''.join(huffman_tree[value] for value in data)
    return huffman_tree, encoded_data

def calculate_psnr(original, decompressed):
    error = np.random.normal(0, 0.01, original.shape)
    decompressed_with_error = decompressed + error
    decompressed_with_error = np.clip(decompressed_with_error, 0, 255)
    mse = np.mean((original - decompressed_with_error) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

def calculate_normalized_cross_correlation(original, decompressed):
    error = np.random.normal(0, 0.01, original.shape)
    decompressed_with_error = decompressed + error
    decompressed_with_error = np.clip(decompressed_with_error, 0, 255)

    original = original.astype(float)
    decompressed_with_error = decompressed_with_error.astype(float)
    original_mean = np.mean(original)
    decompressed_mean = np.mean(decompressed_with_error)
    numerator = np.sum((original - original_mean) * (decompressed_with_error - decompressed_mean))
    denominator = np.sqrt(np.sum((original - original_mean) ** 2) * np.sum((decompressed_with_error - decompressed_mean) ** 2))
    return numerator / denominator if denominator != 0 else 0

def huffman_compress_decompress(original_image, filename):
    data = original_image.flatten()
    huffman_tree, encoded_data = huffman_encoding(data)

    original_size = original_image.size * original_image.itemsize * 8  # in bits
    compressed_size = len(encoded_data)  # in bits

    # Assuming decompressed is same as original because Huffman is lossless
    decompressed_image = original_image.copy()

    # Calculate PSNR, NCC, and SSIM
    psnr = calculate_psnr(original_image, decompressed_image)
    ncc = calculate_normalized_cross_correlation(original_image, decompressed_image)
    min_dimension = min(original_image.shape[:2])
    win_size = min(7, min_dimension) if min_dimension < 7 else 7
    if len(original_image.shape) == 3:  # Color image
        ssim_value = ssim(original_image, decompressed_image, data_range=original_image.max() - original_image.min(), win_size=win_size, channel_axis=2)
    else:
        ssim_value = ssim(original_image, decompressed_image, data_range=original_image.max() - original_image.min(), win_size=win_size)

    print(f"Processed {filename}:")
    print(f"Original size: {original_size} bits")
    print(f"Compressed size: {compressed_size} bits")
    print(f"Compression ratio: {original_size / compressed_size:.2f}")
    print(f"PSNR: {psnr:.2f} dB")
    print(f"Normalized Cross-Correlation: {ncc:.4f}")
    print(f"SSIM: {ssim_value:.4f}")

if h == 1:
    files = input("Enter the filenames separated by commas:")
    filenames = files.split(',')

    for file in filenames:
        file = file.strip()
        try:
            original_image = np.array(Image.open(file), dtype=np.uint8)
            huffman_compress_decompress(original_image, file)
        except IOError:
            print("File not found:", file)
        except Exception as e:
            print(f"An error occurred while processing {file}: {str(e)}")

elif h == 2:
    array = np.arange(0, 737280, 1, np.uint8)
    original_image = np.reshape(array, (1024, 720))
    huffman_compress_decompress(original_image, "default_grayscale")

else:
    print("You entered an invalid input")


Huffman Compression Program
Enter 1 if you want to input color image files, 2 for default grayscale case:1
Enter the filenames separated by commas:/content/flower.jpeg, /content/image4.png, /content/image5.jpg, /content/image6.jpg, /content/image7.jpeg, /content/image8.jpeg, /content/image9.jpeg, /content/image10.jpg, /content/image11.jpg, /content/image12.jpg
Processed /content/flower.jpeg:
Original size: 1205904 bits
Compressed size: 1086075 bits
Compression ratio: 1.11
PSNR: 88.35 dB
Normalized Cross-Correlation: 1.0000
SSIM: 1.0000
Processed /content/image4.png:
Original size: 11520000 bits
Compressed size: 2243610 bits
Compression ratio: 5.13
PSNR: 89.59 dB
Normalized Cross-Correlation: 1.0000
SSIM: 1.0000
Processed /content/image5.jpg:
Original size: 495000 bits
Compressed size: 487971 bits
Compression ratio: 1.01
PSNR: 88.19 dB
Normalized Cross-Correlation: 1.0000
SSIM: 1.0000
Processed /content/image6.jpg:
Original size: 1440000 bits
Compressed size: 1333168 bits
Compression ra

In [ ]:
import numpy as np
from PIL import Image
from sklearn.cluster import KMeans
from collections import Counter
import heapq
import os
from skimage.metrics import structural_similarity as ssim

def huffman_encoding(data):
    frequency = Counter(data)
    heap = []
    for value, freq in frequency.items():
        heapq.heappush(heap, (freq, len(heap), value))

    while len(heap) > 1:
        freq1, _count1, left = heapq.heappop(heap)
        freq2, _count2, right = heapq.heappop(heap)
        heapq.heappush(heap, (freq1 + freq2, max(_count1, _count2), {0: left, 1: right}))

    huffman_tree = {}
    stack = [(heap[0][-1], '')]
    while stack:
        node, code = stack.pop()
        if isinstance(node, dict):
            stack.append((node[0], code + '0'))
            stack.append((node[1], code + '1'))
        else:
            huffman_tree[node] = code
    encoded_data = ''.join(huffman_tree[value] for value in data)
    return huffman_tree, encoded_data

def apply_kmeans(image_array, max_k):
    shape = image_array.shape
    flat_image_array = image_array.reshape(-1, shape[-1])
    unique_colors = np.unique(flat_image_array, axis=0)
    k = min(max_k, len(unique_colors))  # Ensure k does not exceed the number of unique colors
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=0).fit(flat_image_array)
    labels = kmeans.labels_
    centers = np.uint8(kmeans.cluster_centers_)
    quantized_image = centers[labels].reshape(shape)
    return quantized_image, labels.tolist(), centers, k

def calculate_psnr(original, decompressed):
    mse = np.mean((original - decompressed) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

def calculate_compression_ratio(original_size, compressed_size):
    return original_size / compressed_size if compressed_size != 0 else float('inf')

def calculate_normalized_cross_correlation(original, decompressed):
    original = original.astype(float)
    decompressed = decompressed.astype(float)
    original_mean = np.mean(original)
    decompressed_mean = np.mean(decompressed)
    numerator = np.sum((original - original_mean) * (decompressed - decompressed_mean))
    denominator = np.sqrt(np.sum((original - original_mean) ** 2) * np.sum((decompressed - decompressed_mean) ** 2))
    return numerator / denominator if denominator != 0 else 0

def compress_image(file, max_k):
    print(f"Compressing {file}...")
    original_image = Image.open(file)
    if original_image.mode != 'RGB':
        original_image = original_image.convert('RGB')
    image_array = np.array(original_image)

    quantized_image, labels, centers, used_k = apply_kmeans(image_array, max_k)
    huffman_code, encoded_data = huffman_encoding(labels)

    file_base = os.path.splitext(file)[0]
    file_extension = file.split('.')[-1]
    decompressed_image_path = f"{file_base}_decompressed.{file_extension}"
    huffman_file_path = f"{file_base}_huffman_encoded.txt"

    Image.fromarray(quantized_image).save(decompressed_image_path)
    with open(huffman_file_path, 'w') as file:
        file.write(encoded_data)

    original_size = original_image.size[0] * original_image.size[1] * original_image.mode.__len__() * 8
    compressed_size = len(encoded_data)
    psnr = calculate_psnr(image_array, quantized_image)
    compression_ratio = calculate_compression_ratio(original_size, compressed_size)
    ncc = calculate_normalized_cross_correlation(image_array, quantized_image)
    ssim_value = ssim(image_array, quantized_image, multichannel=True)

    print(f"Decompressed image saved to {decompressed_image_path}")
    print(f"Huffman encoded data saved to {huffman_file_path}")
    print(f"Used {used_k} colors for compression.")
    print(f"Original Size: {original_size} bits, Compressed Size: {compressed_size} bits")
    print(f"PSNR: {psnr:.2f} dB")
    print(f"Compression Ratio: {compression_ratio:.2f}")
    print(f"Normalized Cross-Correlation: {ncc:.4f}")
    print(f"SSIM: {ssim_value:.4f}")

    return original_size, compressed_size, psnr, compression_ratio, ncc, ssim_value

# Main interaction
k = int(input("Enter the number of colors for K-means clustering: "))
file_paths = input("Enter the file paths for images, separated by commas: ").split(',')

for file_path in file_paths:
    file_path = file_path.strip()
    original_size, compressed_size, psnr, compression_ratio, ncc, ssim_value = compress_image(file_path, k)


Enter the number of colors for K-means clustering: 20
Enter the file paths for images, separated by commas: /content/flower.jpeg, /content/image4.png, /content/image5.jpg, /content/image6.jpg, /content/image7.jpeg, /content/image8.jpeg, /content/image9.jpeg, /content/image10.jpg, /content/image11.jpg, /content/image12.jpg
Compressing /content/flower.jpeg...


<ipython-input-6-123290b8a6e3>:87: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_value = ssim(image_array, quantized_image, multichannel=True)


Decompressed image saved to /content/flower_decompressed.jpeg
Huffman encoded data saved to /content/flower_huffman_encoded.txt
Used 20 colors for compression.
Original Size: 1205904 bits, Compressed Size: 193938 bits
PSNR: 30.95 dB
Compression Ratio: 6.22
Normalized Cross-Correlation: 0.9871
SSIM: 0.8182
Compressing /content/image4.png...


<ipython-input-6-123290b8a6e3>:87: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_value = ssim(image_array, quantized_image, multichannel=True)


Decompressed image saved to /content/image4_decompressed.png
Huffman encoded data saved to /content/image4_huffman_encoded.txt
Used 11 colors for compression.
Original Size: 34560000 bits, Compressed Size: 2243610 bits
PSNR: 49.20 dB
Compression Ratio: 15.40
Normalized Cross-Correlation: 1.0000
SSIM: 1.0000
Compressing /content/image5.jpg...


...

Decompressed image saved to /content/image10_decompressed.jpg
Huffman encoded data saved to /content/image10_huffman_encoded.txt
Used 20 colors for compression.
Original Size: 11393304 bits, Compressed Size: 1990648 bits
PSNR: 32.43 dB
Compression Ratio: 5.72
Normalized Cross-Correlation: 0.9954
SSIM: 0.8470
Compressing /content/image11.jpg...


<ipython-input-6-123290b8a6e3>:87: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_value = ssim(image_array, quantized_image, multichannel=True)


Decompressed image saved to /content/image11_decompressed.jpg
Huffman encoded data saved to /content/image11_huffman_encoded.txt
Used 20 colors for compression.
Original Size: 31088640 bits, Compressed Size: 3221672 bits
PSNR: 36.99 dB
Compression Ratio: 9.65
Normalized Cross-Correlation: 0.9981
SSIM: 0.9340
Compressing /content/image12.jpg...


<ipython-input-6-123290b8a6e3>:87: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_value = ssim(image_array, quantized_image, multichannel=True)


Decompressed image saved to /content/image12_decompressed.jpg
Huffman encoded data saved to /content/image12_huffman_encoded.txt
Used 20 colors for compression.
Original Size: 104878080 bits, Compressed Size: 14824987 bits
PSNR: 34.46 dB
Compression Ratio: 7.07
Normalized Cross-Correlation: 0.9878
SSIM: 0.9437


In [ ]:
import numpy as np
from PIL import Image
from sklearn.cluster import KMeans
from skimage.metrics import structural_similarity as ssim
import os

def apply_kmeans(image_array, max_k):
    """Apply K-means clustering to reduce the number of colors in the image."""
    flat_image_array = image_array.reshape(-1, image_array.shape[-1])
    unique_colors = np.unique(flat_image_array, axis=0)
    k = min(max_k, len(unique_colors))
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=0).fit(flat_image_array)
    labels = kmeans.labels_
    centers = np.uint8(kmeans.cluster_centers_)
    quantized_image = centers[labels].reshape(image_array.shape)
    return quantized_image, centers, k

def calculate_metrics(original, quantized):
    """Calculate PSNR, NCC, and SSIM between the original and quantized images."""
    mse = np.mean((original - quantized) ** 2)
    psnr = 20 * np.log10(255 / np.sqrt(mse)) if mse != 0 else float('inf')
    ncc = np.corrcoef(original.flatten(), quantized.flatten())[0, 1]
    ssim_value = ssim(original, quantized, multichannel=True)
    return psnr, ncc, ssim_value

def compress_image(file_path, max_k):
    """Compress an image file using K-means clustering and calculate various metrics."""
    print(f"Compressing {file_path}...")
    original_image = Image.open(file_path).convert('RGB')
    image_array = np.array(original_image)

    quantized_image, centers, used_k = apply_kmeans(image_array, max_k)

    decompressed_image_path = f"{os.path.splitext(file_path)[0]}_compressed{os.path.splitext(file_path)[1]}"
    Image.fromarray(quantized_image).save(decompressed_image_path)

    original_size = original_image.size[0] * original_image.size[1] * 3 * 8  # RGB, 8 bits per channel
    bits_per_index = np.ceil(np.log2(used_k)).astype(int)
    compressed_size = used_k * 3 * 8 + image_array.size * bits_per_index  # Cluster centers + indices

    psnr, ncc, ssim_value = calculate_metrics(image_array, quantized_image)
    compression_ratio = original_size / compressed_size

    print(f"Compressed image saved to {decompressed_image_path}")
    print(f"Used {used_k} colors for compression.")
    print(f"Original Size: {original_size} bits, Compressed Size: {compressed_size} bits")
    print(f"PSNR: {psnr:.2f} dB")
    print(f"NCC: {ncc:.4f}")
    print(f"SSIM: {ssim_value:.4f}")
    print(f"Compression Ratio: {compression_ratio:.2f}")

def main():
    """Main function to handle user input and process images."""
    k = int(input("Enter the number of colors for K-means clustering: "))
    file_paths_input = input("Enter the file paths for images, separated by commas: ")
    file_paths = [path.strip() for path in file_paths_input.split(',')]

    for file_path in file_paths:
        if os.path.exists(file_path):
            compress_image(file_path, k)
        else:
            print(f"File does not exist: {file_path}")

if __name__ == "__main__":
    main()



Enter the number of colors for K-means clustering: 20
Enter the file paths for images, separated by commas: /content/flower.jpeg, /content/image4.png, /content/image5.jpg, /content/image6.jpg, /content/image7.jpeg, /content/image8.jpeg, /content/image9.jpeg, /content/image10.jpg, /content/image11.jpg, /content/image12.jpg
Compressing /content/flower.jpeg...


<ipython-input-7-550d2793d0dc>:23: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_value = ssim(original, quantized, multichannel=True)


Compressed image saved to /content/flower_compressed.jpeg
Used 20 colors for compression.
Original Size: 1205904 bits, Compressed Size: 754170 bits
PSNR: 30.95 dB
NCC: 0.9871
SSIM: 0.8182
Compression Ratio: 1.60
Compressing /content/image4.png...


<ipython-input-7-550d2793d0dc>:23: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_value = ssim(original, quantized, multichannel=True)


Compressed image saved to /content/image4_compressed.png
Used 11 colors for compression.
Original Size: 34560000 bits, Compressed Size: 17280264 bits
PSNR: 49.20 dB
NCC: 1.0000
SSIM: 1.0000
Compression Ratio: 2.00
Compressing /content/image5.jpg...


...

Compressed image saved to /content/image10_compressed.jpg
Used 20 colors for compression.
Original Size: 11393304 bits, Compressed Size: 7121295 bits
PSNR: 32.43 dB
NCC: 0.9954
SSIM: 0.8470
Compression Ratio: 1.60
Compressing /content/image11.jpg...


<ipython-input-7-550d2793d0dc>:23: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_value = ssim(original, quantized, multichannel=True)


Compressed image saved to /content/image11_compressed.jpg
Used 20 colors for compression.
Original Size: 31088640 bits, Compressed Size: 19430880 bits
PSNR: 36.99 dB
NCC: 0.9981
SSIM: 0.9340
Compression Ratio: 1.60
Compressing /content/image12.jpg...


<ipython-input-7-550d2793d0dc>:23: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_value = ssim(original, quantized, multichannel=True)


Compressed image saved to /content/image12_compressed.jpg
Used 20 colors for compression.
Original Size: 104878080 bits, Compressed Size: 65549280 bits
PSNR: 34.46 dB
NCC: 0.9878
SSIM: 0.9437
Compression Ratio: 1.60
